In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as SNS
%matplotlib inline

In [2]:
#load datasets
orders = pd.read_csv('data\orders.csv', sep = ';')
items = pd.read_csv('data\items.csv', sep = ';')


In [3]:
#merge datasets on item_code
sales = pd.merge(orders,items,on='item_code',how='left')

In [4]:
#create sales and profit features, change date to datetime datatype
sales['sales'] = sales['unit_price_vat_excl']* sales['quantity']
sales['profit'] = (sales['unit_price_vat_excl'] - sales['unit_cogs'])* sales['quantity']
sales['date']=  pd.to_datetime(sales['date'])

In [5]:
#drop unpaid orders and placeholder items
sales_clean = sales[sales['brand_id']!=-1]
sales_clean = sales_clean[sales_clean['payment'].notna()]

In [30]:
#group sales by period
daily_sales = sales_clean.groupby(sales_clean.date.dt.date)['sales'].sum()
weekly_sales = sales_clean.groupby(pd.Grouper(key='date', freq='W'))['sales'].sum()

In [35]:
#put data in format for prophet
prophet_df= pd.DataFrame()
prophet_df['y'] = daily_sales
prophet_df = prophet_df.reset_index()
prophet_df = prophet_df.rename(columns = {'date':'ds'})
prophet_df.head()

,ds,y
0,2019-06-01,4087.236534
1,2019-06-02,65.848327
2,2019-06-03,1364.979529
3,2019-06-04,68385.521621
4,2019-06-05,65135.657001


In [36]:
prophet_df.to_csv('prophetdf.csv')

In [34]:
#weekly data in format for prophet
wprophet_df= pd.DataFrame()
wprophet_df['y'] = weekly_sales
wprophet_df = wprophet_df.reset_index()
wprophet_df = wprophet_df.rename(columns = {'date':'ds'})
wprophet_df.head()

,ds,y
0,2019-06-02,4153.084860
1,2019-06-09,164184.680219
2,2019-06-16,131655.336749
3,2019-06-23,208592.609565
4,2019-06-30,132990.181248


In [37]:
wprophet_df.to_csv('weekly_prophetdf.csv')

In [ ]:
#group by categories
sales_cat = sales.groupby(['group0_id','date']).sum()
footwear = sales_cat.loc[100]
apparel = sales_cat.loc[200]
equipment = sales_cat.loc[300]
nuitrition = sales_cat.loc[400]